## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-29-18-52-19 +0000,nyt,"South Africans Were Promised Job, but Ended Up...",https://www.nytimes.com/2025/11/29/world/afric...
1,2025-11-29-18-50-11 +0000,nyt,F.D.A. Seeks More Oversight of Vaccine Trials ...,https://www.nytimes.com/2025/11/28/health/fda-...
2,2025-11-29-18-50-00 +0000,nyt,"Tom Stoppard, Award-Winning Playwright of Witt...",https://www.nytimes.com/2025/11/29/theater/tom...
3,2025-11-29-18-49-36 +0000,startribune,"Wintry Minnesota Saturday brings snow, crashes...",https://www.startribune.com/wintry-minnesota-s...
4,2025-11-29-18-47-50 +0000,nypost,Sir Tom Stoppard dead at 88: Legendary playwri...,https://nypost.com/2025/11/29/lifestyle/sir-to...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2329/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
46,trump,37
115,national,12
326,shooting,11
325,guard,10
6,up,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
190,2025-11-28-21-31-00 +0000,wsj,President Trump said he was revoking all execu...,https://www.wsj.com/politics/trump-says-he-is-...,96
128,2025-11-29-06-53-08 +0000,nypost,Trump says he plans to pardon former Honduran ...,https://nypost.com/2025/11/29/us-news/trump-pl...,89
163,2025-11-29-00-35-54 +0000,wapo,Trump plans to pardon former Honduran presiden...,https://www.washingtonpost.com/world/2025/11/2...,78
155,2025-11-29-01-12-00 +0000,wsj,Trump to Pardon Honduran Ex-President Serving ...,https://www.wsj.com/politics/trump-to-pardon-f...,74
34,2025-11-29-16-32-00 +0000,bbc,Trump to pardon ex-Honduras president convicte...,https://www.bbc.com/news/articles/c1kpm0rvxepo...,71


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
190,96,2025-11-28-21-31-00 +0000,wsj,President Trump said he was revoking all execu...,https://www.wsj.com/politics/trump-says-he-is-...
44,68,2025-11-29-15-51-46 +0000,nypost,"Rahmanullah Lakanwal, suspect in deadly Nation...",https://nypost.com/2025/11/29/us-news/rahmanul...
170,44,2025-11-28-23-50-12 +0000,nypost,Treasury Secretary Scott Bessent moves to ‘cut...,https://nypost.com/2025/11/28/us-news/treasury...
37,35,2025-11-29-16-26-44 +0000,nypost,Mom of NJ teen killed outside NYC ax-throwing...,https://nypost.com/2025/11/29/us-news/mom-of-n...
45,34,2025-11-29-15-47-14 +0000,nypost,Ukraine hit two ‘shadow fleet’ oil tankers wit...,https://nypost.com/2025/11/29/world-news/ukrai...
131,34,2025-11-29-05-32-13 +0000,nyt,"Trump to Pardon Juan Orlando Hernández, Hondur...",https://www.nytimes.com/2025/11/28/world/ameri...
142,30,2025-11-29-03-46-48 +0000,nyt,Officials Had Been Warned for Over a Year Befo...,https://www.nytimes.com/2025/11/28/world/asia/...
157,26,2025-11-29-01-04-55 +0000,nypost,"Airbus grounds 6,000 planes for urgent softwar...",https://nypost.com/2025/11/28/world-news/airbu...
49,25,2025-11-29-15-29-44 +0000,nyt,Pope Leo Visits Turkey on His First Trip as Po...,https://www.nytimes.com/2025/11/27/world/europ...
73,25,2025-11-29-12-47-31 +0000,nypost,Mamdani-linked PAC allegedly offered Hispanic ...,https://nypost.com/2025/11/29/us-news/mamdani-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
